In [1]:
import numpy as np 
import pandas as pd  

In [2]:
data = 'C:\\Users\\Lenovo\\Desktop\\college1\\Labs\\semester 5\\Machine Learning\\2101227_Lab_4\\car_evaluation.csv'
df = pd.read_csv(data)

In [3]:
column_names = ['buying price', 'maintenance cost',
                'number of doors', 'number of persons', 'lug_boot', 'safety', 'decision']
df.columns = column_names
df.to_csv('C:\\Users\\Lenovo\\Desktop\\college1\\Labs\\semester 5\\Machine Learning\\2101227_Lab_4\\car_evaluation.csv', index=False)


In [4]:
df

,buying price,maintenance cost,number of doors,number of persons,lug_boot,safety,decision
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc
...,...,...,...,...,...,...,...
1722,low,low,5more,more,med,med,good
1723,low,low,5more,more,med,high,vgood
1724,low,low,5more,more,big,low,unacc
1725,low,low,5more,more,big,med,good


In [5]:
x = df.drop([df.columns[-1]], axis=1)
y = df[df.columns[-1]]

In [6]:
x

,buying price,maintenance cost,number of doors,number of persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,med
1,vhigh,vhigh,2,2,small,high
2,vhigh,vhigh,2,2,med,low
3,vhigh,vhigh,2,2,med,med
4,vhigh,vhigh,2,2,med,high
...,...,...,...,...,...,...
1722,low,low,5more,more,med,med
1723,low,low,5more,more,med,high
1724,low,low,5more,more,big,low
1725,low,low,5more,more,big,med


In [7]:
y

0       unacc
1       unacc
2       unacc
3       unacc
4       unacc
        ...  
1722     good
1723    vgood
1724    unacc
1725     good
1726    vgood
Name: decision, Length: 1727, dtype: object

In [8]:
class NaiveBayes:
    def __init__(self):
        self.features = []  # List of feature names
        self.conditional_probability = {}  # Likelihood of each feature value for each class
        self.class_probability = {}  # Prior probabilities of classes
        self.predictor_probability = {}  # Prior probabilities of feature values
        self.X_train = None 
        self.y_train = None  
        self.train_size = 0  
        self.num_feature = 0 

    def fit(self, X, y):
        # Store the list of feature names
        self.features = list(X.columns)
        self.X_train = X
        self.y_train = y
        self.train_size = X.shape[0]
        self.num_feature = X.shape[1]

        # Initialize dictionaries for conditional proabilty and predictor priors
        for feature in self.features:
            self.conditional_probability[feature] = {}
            self.predictor_probability[feature] = {}
            # Initialize entries for each unique feature value and class
            for feature_val in np.unique(self.X_train[feature]):
                self.predictor_probability[feature].update({feature_val: 0})
                for outcome in np.unique(self.y_train):
                    self.conditional_probability[feature].update({feature_val + '_' + outcome: 0})
                    self.class_probability.update({outcome: 0})
        # Calculate class priors, conditional proabilty, and predictor priors
        self.calculate_class_prior()
        self.calculate_conditional_probability()
        self.calculate_predictor()
        print(f"calculate_conditional_probability : {self.conditional_probability}")
        print(f"calculate_class_prior : {self.class_probability}")
        print(f"calculate_predictor : {self.predictor_probability}")

    def calculate_class_prior(self):
        # Calculate (P(c))
        for i in np.unique(self.y_train):
            count_outcome = sum(self.y_train == i)
            self.class_probability[i] = count_outcome / self.train_size

    def calculate_conditional_probability(self):
        # Calculate (P(x|c))
        for feature in self.features:
            for i in np.unique(self.y_train):
                count_outcome = sum(self.y_train == i)
                feature_conditional_probability = self.X_train[feature][self.y_train[self.y_train == i].index.values.tolist()].value_counts().to_dict()
                for feature_val, count in feature_conditional_probability.items():
                    self.conditional_probability[feature][feature_val + '_' + i] = count / count_outcome

    def calculate_predictor(self):
        # Calculate  (P(x))
        for feature in self.features:
            feature_values = self.X_train[feature].value_counts().to_dict()
            for feature_val, count in feature_values.items():
                self.predictor_probability[feature][feature_val] = count / self.train_size

    def predict(self, X):
        # Calculates probability P(c|x)
        results = []
        X = np.array(X)
        for query in X:
            probs_outcome = {}
            for i in np.unique(self.y_train):
                prior = self.class_probability[i]
                temp = 1
                temp2 = 1
                for feat, feature_val in zip(self.features, query):
                    temp *= self.conditional_probability[feat][feature_val + '_' + i]
                    temp2 *= self.predictor_probability[feat][feature_val]
                pp = (temp * prior) / (temp2)
                probs_outcome[i] = pp
            result = max(probs_outcome, key=lambda x: probs_outcome[x])
            results.append(result)
        return np.array(results)

In [9]:
def accuracy_score(y_true, y_pred):
	# score = (y_test - y_pred) / len(y_test)
	return round(float(sum(y_pred == y_true))/float(len(y_true)) * 100, 2)

In [10]:
nb = NaiveBayes()
nb.fit(x, y)

calculate_conditional_probability : {'buying price': {'high_acc': 0.28125, 'high_good': 0, 'high_unacc': 0.2679900744416873, 'high_vgood': 0, 'low_acc': 0.23177083333333334, 'low_good': 0.6666666666666666, 'low_unacc': 0.21339950372208435, 'low_vgood': 0.6, 'med_acc': 0.2994791666666667, 'med_good': 0.3333333333333333, 'med_unacc': 0.22167080231596362, 'med_vgood': 0.4, 'vhigh_acc': 0.1875, 'vhigh_good': 0, 'vhigh_unacc': 0.2969396195202647, 'vhigh_vgood': 0}, 'maintenance cost': {'high_acc': 0.2734375, 'high_good': 0, 'high_unacc': 0.2597187758478081, 'high_vgood': 0.2, 'low_acc': 0.23958333333333334, 'low_good': 0.6666666666666666, 'low_unacc': 0.22167080231596362, 'low_vgood': 0.4, 'med_acc': 0.2994791666666667, 'med_good': 0.3333333333333333, 'med_unacc': 0.22167080231596362, 'med_vgood': 0.4, 'vhigh_acc': 0.1875, 'vhigh_good': 0, 'vhigh_unacc': 0.2969396195202647, 'vhigh_vgood': 0}, 'number of doors': {'2_acc': 0.2109375, '2_good': 0.21739130434782608, '2_unacc': 0.268817204301075

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=10)
print(f"Train Accuracy:{accuracy_score(y_test, nb.predict(X_test))}")

Train Accuracy:90.75


In [12]:
y_pred = nb.predict(X_test)
y_pred

array(['unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc',
       'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'acc', 'acc', 'acc',
       'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc',
       'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc',
       'unacc', 'acc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc',
       'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'vgood',
       'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'acc', 'unacc',
       'acc', 'vgood', 'good', 'unacc', 'acc', 'acc', 'unacc', 'unacc',
       'unacc', 'unacc', 'unacc', 'acc', 'acc', 'unacc', 'unacc', 'acc',
       'unacc', 'unacc', 'vgood', 'unacc', 'unacc', 'unacc', 'acc',
       'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'vgood',
       'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'acc', 'acc', 'acc',
       'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc',
       'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc',
       'vgood', 'un

In [13]:
y_test

954     unacc
115     unacc
1463    unacc
35      unacc
1625    unacc
        ...  
951     unacc
981     unacc
191     unacc
1635     good
778       acc
Name: decision, Length: 173, dtype: object

In [14]:
query = np.array([['low', 'low', '5more', '4', 'small', 'high']])
print(f"Query : {query} = {nb.predict(query)}")

Query : [['low' 'low' '5more' '4' 'small' 'high']] = ['good']
